<a href="https://colab.research.google.com/github/ved1beta/Triton/blob/main/falseattention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install torch triton numpy matplotlib datasets transformers seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
import torch
import triton
import triton.language as tl
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import BertTokenizer
import time
from torch.utils.data import DataLoader
import seaborn as sns
from tqdm import tqdm

# Import the Triton attention implementation from the original code
@triton.jit
def attention_qk_kernel(
    q_ptr, k_ptr, output_ptr,
    seq_length, head_size,
    BLOCK_SIZE: tl.constexpr
):
    pid = tl.program_id(0)
    row_idx = pid // (seq_length // BLOCK_SIZE)
    col_idx = pid % (seq_length // BLOCK_SIZE)

    q_start = q_ptr + row_idx * BLOCK_SIZE * head_size
    k_start = k_ptr + col_idx * BLOCK_SIZE * head_size

    acc = tl.zeros((BLOCK_SIZE, BLOCK_SIZE), dtype=tl.float32)

    for i in range(0, head_size, BLOCK_SIZE):
        q_block = tl.load(q_start + i)
        k_block = tl.load(k_start + i)
        acc += tl.dot(q_block, tl.trans(k_block))

    acc = acc / tl.sqrt(float(head_size))

    output_start = output_ptr + row_idx * BLOCK_SIZE * seq_length + col_idx * BLOCK_SIZE
    tl.store(output_start, acc)

@triton.jit
def softmax_kernel(
    ptr_in, ptr_out,
    n_cols,
    BLOCK_SIZE: tl.constexpr
):
    row_idx = tl.program_id(0)
    row_start = ptr_in + row_idx * n_cols

    row = tl.load(row_start + tl.arange(0, n_cols))
    row_max = tl.max(row, axis=0)
    numerator = tl.exp(row - row_max)
    denominator = tl.sum(numerator, axis=0)
    softmax_output = numerator / denominator

    tl.store(ptr_out + row_idx * n_cols + tl.arange(0, n_cols), softmax_output)

class TritonAttention(torch.nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size

    def forward(self, q, k, v):
        batch_size = q.shape[0]
        seq_len = q.shape[1]

        grid = (batch_size * seq_len * seq_len) // 256

        attention_scores = torch.empty(
            (batch_size, seq_len, seq_len),
            device='cuda',
            dtype=torch.float32
        )

        attention_qk_kernel[grid](
            q.data_ptr(),
            k.data_ptr(),
            attention_scores.data_ptr(),
            seq_len,
            self.head_size,
            BLOCK_SIZE=16
        )

        attention_probs = torch.empty_like(attention_scores)
        softmax_kernel[grid](
            attention_scores.data_ptr(),
            attention_probs.data_ptr(),
            seq_len,
            BLOCK_SIZE=256
        )

        output = torch.empty_like(q)
        attention_qk_kernel[grid](
            attention_probs.data_ptr(),
            v.data_ptr(),
            output.data_ptr(),
            seq_len,
            self.head_size,
            BLOCK_SIZE=16
        )

        return output

def load_and_prepare_data(batch_size=32, max_length=256):
    """Load and prepare the Wikipedia dataset"""
    print("Loading dataset...")
    dataset = load_dataset("wikipedia", "20220301.en", split="train[:1000]")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            padding='max_length',
            truncation=True,
            max_length=max_length
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    dataloader = DataLoader(
        tokenized_dataset,
        batch_size=batch_size,
        shuffle=True
    )

    return dataloader

def run_attention_comparison(dataloader, head_size=64, num_batches=10):
    """Run comparison between PyTorch and Triton attention"""
    triton_attention = TritonAttention(head_size).cuda()

    triton_times = []
    pytorch_times = []
    accuracy_diffs = []

    print("Running attention comparison...")
    for batch in tqdm(list(dataloader)[:num_batches]):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']

        # Create random Q, K, V tensors based on the input shape
        batch_size = input_ids.shape[0]
        seq_len = input_ids.shape[1]

        q = torch.randn(batch_size, seq_len, head_size, device='cuda')
        k = torch.randn(batch_size, seq_len, head_size, device='cuda')
        v = torch.randn(batch_size, seq_len, head_size, device='cuda')

        # Warm-up run
        _ = triton_attention(q, k, v)
        _ = torch.nn.functional.scaled_dot_product_attention(q, k, v)

        # Measure Triton performance
        torch.cuda.synchronize()
        start_time = time.time()
        triton_output = triton_attention(q, k, v)
        torch.cuda.synchronize()
        triton_time = time.time() - start_time
        triton_times.append(triton_time)

        # Measure PyTorch performance
        torch.cuda.synchronize()
        start_time = time.time()
        pytorch_output = torch.nn.functional.scaled_dot_product_attention(q, k, v)
        torch.cuda.synchronize()
        pytorch_time = time.time() - start_time
        pytorch_times.append(pytorch_time)

        # Calculate accuracy difference
        max_diff = torch.max(torch.abs(triton_output - pytorch_output))
        accuracy_diffs.append(max_diff.item())

    return triton_times, pytorch_times, accuracy_diffs

def plot_results(triton_times, pytorch_times, accuracy_diffs):
    """Plot performance comparison results"""
    plt.figure(figsize=(15, 5))

    # Performance comparison
    plt.subplot(1, 2, 1)
    data = {
        'Triton': triton_times,
        'PyTorch': pytorch_times
    }
    sns.boxplot(data=data)
    plt.title('Attention Implementation Performance Comparison')
    plt.ylabel('Time (seconds)')

    # Accuracy differences
    plt.subplot(1, 2, 2)
    plt.plot(accuracy_diffs, marker='o')
    plt.title('Maximum Difference between Implementations')
    plt.xlabel('Batch')
    plt.ylabel('Maximum Absolute Difference')

    plt.tight_layout()
    plt.savefig('attention_comparison_results.png')
    print("Results saved to attention_comparison_results.png")

def main():
    # Configuration
    batch_size = 32
    max_length = 256
    head_size = 64
    num_batches = 10

    # Load data
    dataloader = load_and_prepare_data(batch_size, max_length)

    # Run comparison
    triton_times, pytorch_times, accuracy_diffs = run_attention_comparison(
        dataloader,
        head_size,
        num_batches
    )

    # Plot and save results
    plot_results(triton_times, pytorch_times, accuracy_diffs)

    # Print summary statistics
    print("\nPerformance Summary:")
    print(f"Average Triton Time: {np.mean(triton_times):.4f} seconds")
    print(f"Average PyTorch Time: {np.mean(pytorch_times):.4f} seconds")
    print(f"Average Maximum Difference: {np.mean(accuracy_diffs):.6f}")
    print(f"Speedup: {np.mean(pytorch_times)/np.mean(triton_times):.2f}x")

if __name__ == "__main__":
    main()

Loading dataset...
Running attention comparison...


  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: only integer tensors of a single element can be converted to an index

In [9]:
import torch
import time
from tqdm import tqdm
from datasets import load_dataset
from transformers import BertTokenizer
from torch.utils.data import DataLoader
import triton
import triton.language as tl

@triton.jit
def attention_qk_kernel(
    q_ptr, k_ptr, output_ptr,
    seq_length, head_size,
    BLOCK_SIZE: tl.constexpr
):
    pid = tl.program_id(0)
    row_idx = pid // (seq_length // BLOCK_SIZE)
    col_idx = pid % (seq_length // BLOCK_SIZE)

    q_start = q_ptr + row_idx * BLOCK_SIZE * head_size
    k_start = k_ptr + col_idx * BLOCK_SIZE * head_size

    acc = tl.zeros((BLOCK_SIZE, BLOCK_SIZE), dtype=tl.float32)

    for i in range(0, head_size, BLOCK_SIZE):
        q_block = tl.load(q_start + i)
        k_block = tl.load(k_start + i)
        acc += tl.dot(q_block, tl.trans(k_block))

    acc = acc / tl.sqrt(float(head_size))

    output_start = output_ptr + row_idx * BLOCK_SIZE * seq_length + col_idx * BLOCK_SIZE
    tl.store(output_start, acc)

@triton.jit
def softmax_kernel(
    ptr_in, ptr_out,
    n_cols,
    BLOCK_SIZE: tl.constexpr
):
    row_idx = tl.program_id(0)
    row_start = ptr_in + row_idx * n_cols

    row = tl.load(row_start + tl.arange(0, n_cols))
    row_max = tl.max(row, axis=0)
    numerator = tl.exp(row - row_max)
    denominator = tl.sum(numerator, axis=0)
    softmax_output = numerator / denominator

    tl.store(ptr_out + row_idx * n_cols + tl.arange(0, n_cols), softmax_output)

class TritonAttention(torch.nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size

    def forward(self, q, k, v):
        batch_size = q.shape[0]
        seq_len = q.shape[1]

        grid = (batch_size * seq_len * seq_len) // 256

        attention_scores = torch.empty(
            (batch_size, seq_len, seq_len),
            device='cuda',
            dtype=torch.float32
        )

        attention_qk_kernel[grid](
            q.data_ptr(),
            k.data_ptr(),
            attention_scores.data_ptr(),
            seq_len,
            self.head_size,
            BLOCK_SIZE=16
        )

        attention_probs = torch.empty_like(attention_scores)
        softmax_kernel[grid](
            attention_scores.data_ptr(),
            attention_probs.data_ptr(),
            seq_len,
            BLOCK_SIZE=256
        )

        output = torch.empty_like(q)
        attention_qk_kernel[grid](
            attention_probs.data_ptr(),
            v.data_ptr(),
            output.data_ptr(),
            seq_len,
            self.head_size,
            BLOCK_SIZE=16
        )

        return output

def load_and_prepare_data(batch_size=32, max_length=256):
    """Load and prepare the Wikipedia dataset"""
    print("Loading dataset...")
    dataset = load_dataset("wikipedia", "20220301.en", split="train[:1000]")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            padding='max_length',
            truncation=True,
            max_length=max_length
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    dataloader = DataLoader(
        tokenized_dataset,
        batch_size=batch_size,
        shuffle=True
    )

    return dataloader

def run_attention_comparison(dataloader, head_size=64, num_batches=10):
    """Run comparison between PyTorch and Triton attention"""
    triton_attention = TritonAttention(head_size).cuda()

    triton_times = []
    pytorch_times = []
    accuracy_diffs = []

    print("Running attention comparison...")
    for batch in tqdm(list(dataloader)[:num_batches]):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']

        # Ensure input_ids and attention_mask are tensors
        if isinstance(input_ids, list):
            input_ids = torch.tensor(input_ids, device='cuda')
        if isinstance(attention_mask, list):
            attention_mask = torch.tensor(attention_mask, device='cuda')

        # Create random Q, K, V tensors based on the input shape
        batch_size = input_ids.shape[0]
        seq_len = input_ids.shape[1]

        q = torch.randn(batch_size, seq_len, head_size, device='cuda')
        k = torch.randn(batch_size, seq_len, head_size, device='cuda')
        v = torch.randn(batch_size, seq_len, head_size, device='cuda')

        # Warm-up run
        _ = triton_attention(q, k, v)
        _ = torch.nn.functional.scaled_dot_product_attention(q, k, v)

        # Measure Triton performance
        torch.cuda.synchronize()
        start_time = time.time()
        triton_output = triton_attention(q, k, v)
        torch.cuda.synchronize()
        triton_time = time.time() - start_time
        triton_times.append(triton_time)

        # Measure PyTorch performance
        torch.cuda.synchronize()
        start_time = time.time()
        pytorch_output = torch.nn.functional.scaled_dot_product_attention(q, k, v)
        torch.cuda.synchronize()
        pytorch_time = time.time() - start_time
        pytorch_times.append(pytorch_time)

        # Calculate accuracy difference
        max_diff = torch.max(torch.abs(triton_output - pytorch_output))
        accuracy_diffs.append(max_diff.item())

    return triton_times, pytorch_times, accuracy_diffs

if __name__ == "__main__":
    dataloader = load_and_prepare_data()
    triton_times, pytorch_times, accuracy_diffs = run_attention_comparison(dataloader)
    print("Triton times:", triton_times)
    print("PyTorch times:", pytorch_times)
    print("Accuracy differences:", accuracy_diffs)


Loading dataset...
Running attention comparison...


  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: only integer tensors of a single element can be converted to an index

In [21]:
import torch
import time
from tqdm import tqdm
from datasets import load_dataset
from transformers import BertTokenizer
from torch.utils.data import DataLoader
import triton
import triton.language as tl

@triton.jit
def attention_qk_kernel(
    q_ptr, k_ptr, output_ptr,
    seq_length, head_size,
    BLOCK_SIZE: tl.constexpr
):
    # Compute program ID
    pid = tl.program_id(0)
    row_idx = pid // (seq_length // BLOCK_SIZE)
    col_idx = pid % (seq_length // BLOCK_SIZE)

    # Compute pointer offsets for Q and K (without casting to integer types)
    q_start = q_ptr + row_idx * BLOCK_SIZE * head_size
    k_start = k_ptr + col_idx * BLOCK_SIZE * head_size

    # Accumulator for the attention scores
    acc = tl.zeros((BLOCK_SIZE, BLOCK_SIZE), dtype=tl.float32)

    # Iterate over head_size in BLOCK_SIZE chunks
    for i in range(0, head_size, BLOCK_SIZE):
        # Compute chunk offsets for Q and K
        q_chunk = q_start + i
        k_chunk = k_start + i

        # Load Q and K blocks
        q_block = tl.load(q_chunk, mask=None, other=0.0)
        k_block = tl.load(k_chunk, mask=None, other=0.0)

        # Compute dot product and accumulate the result
        acc += tl.dot(q_block, tl.trans(k_block))

    # Normalize the attention scores
    scale = tl.sqrt(float(head_size))
    acc = acc / scale

    # Compute the output pointer offset
    output_start = output_ptr + row_idx * BLOCK_SIZE * seq_length + col_idx * BLOCK_SIZE

    # Store the result
    tl.store(output_start, acc)



@triton.jit
def softmax_kernel(
    ptr_in, ptr_out,
    n_cols,
    BLOCK_SIZE: tl.constexpr
):
    row_idx = tl.program_id(0)
    row_start = ptr_in + row_idx * n_cols

    row = tl.load(row_start + tl.arange(0, n_cols))
    row_max = tl.max(row, axis=0)
    numerator = tl.exp(row - row_max)
    denominator = tl.sum(numerator, axis=0)
    softmax_output = numerator / denominator

    tl.store(ptr_out + row_idx * n_cols + tl.arange(0, n_cols), softmax_output)

class TritonAttention(torch.nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size

    def forward(self, q, k, v):
        batch_size = q.shape[0]
        seq_len = q.shape[1]

        grid = (batch_size * seq_len * seq_len) // 256

        attention_scores = torch.empty(
            (batch_size, seq_len, seq_len),
            device='cuda',
            dtype=torch.float32
        )

        attention_qk_kernel[grid](
            q.data_ptr(),
            k.data_ptr(),
            attention_scores.data_ptr(),
            seq_len,
            self.head_size,
            BLOCK_SIZE=16
        )

        attention_probs = torch.empty_like(attention_scores)
        softmax_kernel[grid](
            attention_scores.data_ptr(),
            attention_probs.data_ptr(),
            seq_len,
            BLOCK_SIZE=256
        )

        output = torch.empty_like(q)
        attention_qk_kernel[grid](
            attention_probs.data_ptr(),
            v.data_ptr(),
            output.data_ptr(),
            seq_len,
            self.head_size,
            BLOCK_SIZE=16
        )

        return output

def load_and_prepare_data(batch_size=32, max_length=256):
    """Load and prepare the Wikipedia dataset"""
    print("Loading dataset...")
    dataset = load_dataset("wikipedia", "20220301.en", split="train[:1000]")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            padding='max_length',
            truncation=True,
            max_length=max_length
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    dataloader = DataLoader(
        tokenized_dataset,
        batch_size=batch_size,
        shuffle=True
    )

    return dataloader

def run_attention_comparison(dataloader, head_size=64, num_batches=10):
    """Run comparison between PyTorch and Triton attention"""
    triton_attention = TritonAttention(head_size).cuda()

    triton_times = []
    pytorch_times = []
    accuracy_diffs = []

    print("Running attention comparison...")
    for batch in tqdm(list(dataloader)[:num_batches]):
        print(batch)  # Debugging: Check the batch structure
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']

        # Convert to PyTorch tensors if necessary
        if isinstance(input_ids, list):
            input_ids = torch.stack([torch.tensor(x, dtype=torch.long) for x in input_ids], dim=0).to('cuda')
        elif isinstance(input_ids, torch.Tensor):
            input_ids = input_ids.to('cuda')
        else:
            raise ValueError("Unexpected format for input_ids")

        if isinstance(attention_mask, list):
            attention_mask = torch.stack([torch.tensor(x, dtype=torch.long) for x in attention_mask], dim=0).to('cuda')
        elif isinstance(attention_mask, torch.Tensor):
            attention_mask = attention_mask.to('cuda')
        else:
            raise ValueError("Unexpected format for attention_mask")

        # Create random Q, K, V tensors based on the input shape
        batch_size = input_ids.shape[0]
        seq_len = input_ids.shape[1]

        q = torch.randn(batch_size, seq_len, head_size, device='cuda')
        k = torch.randn(batch_size, seq_len, head_size, device='cuda')
        v = torch.randn(batch_size, seq_len, head_size, device='cuda')

        # Warm-up run
        _ = triton_attention(q, k, v)
        _ = torch.nn.functional.scaled_dot_product_attention(q, k, v)

        # Measure Triton performance
        torch.cuda.synchronize()
        start_time = time.time()
        triton_output = triton_attention(q, k, v)
        torch.cuda.synchronize()
        triton_time = time.time() - start_time
        triton_times.append(triton_time)

        # Measure PyTorch performance
        torch.cuda.synchronize()
        start_time = time.time()
        pytorch_output = torch.nn.functional.scaled_dot_product_attention(q, k, v)
        torch.cuda.synchronize()
        pytorch_time = time.time() - start_time
        pytorch_times.append(pytorch_time)

        # Calculate accuracy difference
        max_diff = torch.max(torch.abs(triton_output - pytorch_output))
        accuracy_diffs.append(max_diff.item())

    return triton_times, pytorch_times, accuracy_diffs


if __name__ == "__main__":
    dataloader = load_and_prepare_data()
    triton_times, pytorch_times, accuracy_diffs = run_attention_comparison(dataloader)
    print("Triton times:", triton_times)
    print("PyTorch times:", pytorch_times)
    print("Accuracy differences:", accuracy_diffs)


Loading dataset...
Running attention comparison...


  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-21-b534edf3c959>:155: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.stack([torch.tensor(x, dtype=torch.long) for x in input_ids], dim=0).to('cuda')
<ipython-input-21-b534edf3c959>:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.stack([torch.tensor(x, dtype=torch.long) for x in attention_mask], dim=0).to('cuda')
  0%|          | 0/10 [00:00<?, ?it/s]

{'id': ['2391', '1967', '1597', '2047', '1394', '2102', '1710', '801', '1939', '1719', '2440', '38649019', '1600', '2257', '905', '1628', '2130', '1661', '925', '1941', '1484', '681', '802', '864', '1937', '2310', '1596', '1445', '901', '1022', '1217', '1592'], 'url': ['https://en.wikipedia.org/wiki/Akio%20Morita', 'https://en.wikipedia.org/wiki/Apollo%2012', 'https://en.wikipedia.org/wiki/Alexander%20of%20Greece%20%28disambiguation%29', 'https://en.wikipedia.org/wiki/Alcoholics%20Anonymous', 'https://en.wikipedia.org/wiki/Algol', 'https://en.wikipedia.org/wiki/Arizona%20Cardinals', 'https://en.wikipedia.org/wiki/April%2022', 'https://en.wikipedia.org/wiki/Asterism', 'https://en.wikipedia.org/wiki/Approximant', 'https://en.wikipedia.org/wiki/Ambrosius%20Aurelianus', 'https://en.wikipedia.org/wiki/Albrecht%20Altdorfer', 'https://en.wikipedia.org/wiki/Conservative%20Presbyterian%20Church%20in%20Brazil', 'https://en.wikipedia.org/wiki/Severus%20Alexander', 'https://en.wikipedia.org/wiki/A

CompilationError: at 25:18:

    # Accumulator for the attention scores
    acc = tl.zeros((BLOCK_SIZE, BLOCK_SIZE), dtype=tl.float32)

    # Iterate over head_size in BLOCK_SIZE chunks
    for i in range(0, head_size, BLOCK_SIZE):
        # Compute chunk offsets for Q and K
        q_chunk = q_start + i
        k_chunk = k_start + i

        # Load Q and K blocks
        q_block = tl.load(q_chunk, mask=None, other=0.0)
                  ^